## Day 29 Lecture 2 Assignment

In this assignment, we will learn about entropy and information gain in the ID3 algorithm.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
tennis = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/tennis_decision.csv')

In [ ]:
tennis.count().T

Day         14
Outlook     14
Temp.       14
Humidity    14
Wind        14
Decision    14
dtype: int64

Write a function to compute entropy given an input of a sequence of probabilities.

In [ ]:
#answer below:

from math import log2

# EXAMPLE SOLUTION
def entropy_calc(df):
  
  dfSum = df['Yes'].sum()
  dfTrue = dfSum
  #probability of true
  prob_yes =  df['Yes']['Overcast'] / dfSum * _
  log2(df['Yes']['Overcast'] / dfSum) + df['Yes']['Sunny'] / dfSum * log2(dfSum / df['Yes']['Sunny']) + df['Yes']['Rain'] / dfSum * log2(dfSum / df['Yes']['Rain'] )
  
  dfDiff = df['No'].sum()
  #probability of false
  prob_not = dfTrue / dfDiff * log2(dfDiff / dfTrue) + df['Yes']['Sunny'] / dfDiff * log2(dfDiff / df['Yes']['Sunny'] + df['Yes']['Rain']) / dfDiff * log2(dfDiff / df['Yes']['Rain'] )

  tot = dfSum + dfDiff
  entropy = dfSum / tot * prob_yes + dfSum / dfDiff * prob_not
  return entropy 


Aggregate the tennis decision table for each value of each column. Start with Outlook below. Compute the weighted mean of the entropy for outlook (the weighted mean of the yes decision and the no decision).

In [ ]:
# answer below:
a = pd.crosstab(tennis['Outlook'], tennis['Decision'])
print(entropy_calc(a))

Decision        No       Yes
Outlook                     
Overcast -2.486508 -1.343650
Rain     -1.915079 -1.629365
Sunny    -1.629365 -1.915079


In [ ]:
ID3_entropies(a)

No     0.918296
Yes    0.918296
dtype: float64

Compute the weighted mean of the entropy for temperature, humidity and wind as well and decide based on these values which should be the first variable chosen for a split.

The Day should be the first node for our tree. 

In [ ]:
entropy_df = tennis.drop(columns='Decision')
ID3_entropies(entropy_df)

Day         0.371232
Outlook     0.918239
Temp.       0.915452
Humidity    1.000000
Wind        0.985228
dtype: float64

In [ ]:
# Github ID3_entropies retrieved from: https://gist.github.com/whitehaven/bbd408edca38de93637635b52d2bba89
from collections import Counter

def ID3_entropies(data_df):
    """
    Takes pandas.DataFrame and returns a series with all non-index schemas' entropies calculated.
    
    It supports non-binary field types by calculating average entropy. Result series starts with the most productive decision level.
    """
    def entropy_for_field(field):
        entropy = 0
        field_entry_count = len(field)

        # get count of unique
        field_counter = Counter(field)

        # E( Si/S * E(pi*log2(pi)) )
        for trait, count in field_counter.items():
            p_T = count / field_entry_count
            p_F = (field_entry_count - count) / field_entry_count

            if p_T == 0 or p_F == 0:
                entropy = 0
                break
            # Si/S * E(pi*log2(pi))
            entropy += count / field_entry_count * (- (p_T * log2(p_T)) - (p_F * log2(p_F)))
        return entropy
    
    data_df_entropy = {}
    for field in data_df:
        entropy_this_field = entropy_for_field(data_df[field])
        data_df_entropy[field] = entropy_this_field

    data_df_entropy_se = pd.Series(data_df_entropy)
    #data_df_entropy_se.sort(inplace=True)
    return data_df_entropy_se